# GoodHunt v3+ RL Trading Bot — Colab Demo

Profit-optimized RL framework for multi-asset trading. 

Covers: Data, Feature Engineering, Training, Backtest, Plots, Configs.

*(Requires Python 3.9+ and Stable Baselines3)*

In [ ]:
# ⬇️ Install dependencies
!pip install yfinance pandas numpy matplotlib stable-baselines3 gymnasium pyyaml

In [ ]:
# ⬇️ Clone and Setup (if using GitHub repo)
# !git clone https://github.com/YOURUSER/goodhunt-v3.git
# %cd goodhunt-v3

# Or upload code files below if running standalone.

In [ ]:
# ⚙️ Config YAML (edit here or upload your own)
import yaml
config = {
    'data': {'assets': ['AAPL'], 'interval': '1d', 'start': '2023-01-01', 'end': '2024-06-30'},
    'env': {'initial_balance': 100.0, 'window_size': 50, 'fee_pct': 0.001, 'slippage_pct': 0.0015,
            'liquidity_threshold': 1000, 'position_sizing': 'volatility', 'exposure_cap': 0.7,
            'profit_target': 0.05, 'stop_loss': 0.03, 'max_drawdown': 0.05},
    'reward': {'mode': 'sharpe', 'decay': 0.995, 'win_boost': 0.01, 'hold_penalty': 0.01,
               'chaos_flat_bonus': 0.02, 'reward_clip': 2.0},
    'agent': {'algo': 'PPO', 'total_timesteps': 10000, 'policy': 'MlpPolicy',
              'tensorboard_log': 'tensorboard', 'multi_agent': False},
    'logging': {'log_dir': 'logs', 'trade_log': 'backtest/trades.csv',
                'equity_curve': 'backtest/equity_curve.csv', 'plot_dir': 'backtest/plots', 'tensorboard': True},
    'backtest': {'test_split': 0.2, 'multi_asset': False, 'plot_equity_curve': True, 'plot_confusion_matrix': True}
}
with open('utils/config.yaml', 'w') as f:
    yaml.dump(config, f)
print('Config written to utils/config.yaml')

## 1. Download & Preprocess Data

In [ ]:
from data.fetch_data import get_data
from utils.indicators import add_all_indicators
from utils.patterns import add_patterns
from utils.regime import detect_regime

df = get_data(symbol='AAPL', start='2023-01-01', end='2024-06-30', interval='1d', preprocess=False, save=False)
df = add_all_indicators(df)
df = add_patterns(df)
df = detect_regime(df)

df.tail()

## 2. Train RL Agent

In [ ]:
from env.trading_env import TradingEnv
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

env = DummyVecEnv([lambda: TradingEnv(df)])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log="tensorboard")
model.learn(total_timesteps=10000)
model.save('models/demo_ppo_model')
print('Model trained and saved.')

## 3. Backtest & Evaluate

In [ ]:
from env.trading_env import TradingEnv
model = PPO.load('models/demo_ppo_model')
env = TradingEnv(df)
obs, _ = env.reset()
done = False
while not done:
    action, _ = model.predict(obs)
    obs, reward, done, _, _ = env.step(action)
env.save_trades('backtest/trades.csv')
env.save_equity_curve('backtest/equity_curve.csv')
print('Backtest done. Logs saved.')

## 4. Visualize Results

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

eq = pd.read_csv('backtest/equity_curve.csv')
plt.figure(figsize=(10,4))
plt.plot(eq['net_worth'])
plt.title('Equity Curve')
plt.xlabel('Step')
plt.ylabel('Net Worth')
plt.show()

trades = pd.read_csv('backtest/trades.csv')
plt.figure(figsize=(10,4))
plt.scatter(trades['step'], trades['pnl'], c=(trades['pnl']>0), cmap='bwr', label='Trade PnL')
plt.title('Trade PnL by Step')
plt.xlabel('Step')
plt.ylabel('PnL')
plt.axhline(0, color='gray', linestyle='--')
plt.show()

## 5. Config & Advanced Experiments

- Edit `utils/config.yaml` and re-run cells to tune reward, agent, or env.
- Try multi-asset: set `assets: ['AAPL', 'TSLA']` in config and retrain.
- Use `main.py` for advanced CLI (train/test/grid).